# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import os

# Import API key
from api_keys import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
def generate_city_list_csv():    
    # Lists for holding lat_lngs and cities
    lat_lngs = []
    cities = []
    countries = []
    latitudes = []
    longitudes = []
    weather_urls = []
    
    weather_base_url = "http://api.openweathermap.org/data/2.5/weather?appid=" + api_key + "&units=imperial"

    # Create a set of random lat and lng combinations
    lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
    lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
    lat_lngs = zip(lats, lngs)

    # Identify nearest city for each lat, lng combination
    for lat_lng in lat_lngs:
        city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
        weather_url = weather_base_url + "&q=" + city
        city_country = citipy.nearest_city(lat_lng[0], lat_lng[1]).country_code
        city_lat = lat_lng[0]
        city_lng = lat_lng[1]
    
        # If the city is unique, then add it to a our cities list
        if city not in cities:
            cities.append(city)
            countries.append(city_country.upper())
            latitudes.append(city_lat)
            longitudes.append(city_lng)
            weather_urls.append(weather_url)

    # Print the city count to confirm sufficient count
    print(len(cities))

    city_country_coord_url = zip(cities, countries, latitudes, longitudes, weather_urls)

    cities_df = pd.DataFrame(city_country_coord_url, columns=["City", "Country Code", "Latitude", "Longitude", "URL"])
    cities_df.head()

    country_xref_csv = os.path.join("Resources", "country_code_2char.csv")

    country_xref_df = pd.read_csv(country_xref_csv)
    country_xref_df = country_xref_df.rename(columns={"Code": "Country Code"})
    country_xref_df.head()

    city_country_df = pd.merge(cities_df, country_xref_df, on="Country Code")
    city_country_df

    city_frozen_csv = os.path.join("Resources", "Cities_Input_Dataset.csv")
    city_country_df.to_csv(city_frozen_csv, index=False)

In [3]:
# See function above for how I got my set of cities
# I just wanted a static set of cities, not different cities every run
#generate_city_list_csv()

# with set of cities already generared, just read from that csv
my_city_csv = os.path.join("Resources", "Cities_Input_Dataset.csv")
my_city_df = pd.read_csv(my_city_csv)

my_city_df.count()
my_city_df.head()

,City,Country Code,Latitude,Longitude,URL,Name
0,bismil,TR,38.220664,40.589828,http://api.openweathermap.org/data/2.5/weather...,Turkey
1,kurtalan,TR,37.829107,41.667277,http://api.openweathermap.org/data/2.5/weather...,Turkey
2,sinop,TR,42.671415,35.104039,http://api.openweathermap.org/data/2.5/weather...,Turkey
3,akdagmadeni,TR,39.236351,36.320491,http://api.openweathermap.org/data/2.5/weather...,Turkey
4,solhan,TR,39.141941,41.239665,http://api.openweathermap.org/data/2.5/weather...,Turkey


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [4]:
def get_weather_data(city_input_df):
    city_temps = []
    city_humidities = []
    city_clouds = []
    city_winds = []
    city_dates = []
    
    for index, row in city_input_df.iterrows():
        query_url = row["URL"]
        
        weather_data = requests.get(query_url).json()
        
        try:
            city_temp = weather_data["main"]["temp_max"]
            city_temps.append(city_temp)
        
            city_humidity = weather_data["main"]["humidity"]
            city_humidities.append(city_humidity)
        
            city_cloud = weather_data["clouds"]["all"]
            city_clouds.append(city_cloud)
        
            city_wind = weather_data["wind"]["speed"]
            city_winds.append(city_wind)
            
            city_date = weather_data["dt"]
            city_dates.append(city_date)
        except KeyError:
            city_temp = None
            city_temps.append(city_temp)
        
            city_humidity = None
            city_humidities.append(city_humidity)
        
            city_cloud = None
            city_clouds.append(city_cloud)
        
            city_wind = None
            city_winds.append(city_wind)
            
            city_date = None
            city_dates.append(city_date)
            
        #print(f"{city_temp} {city_humidity} {city_cloud} {city_wind} {city_date}")
                
    city_input_df["Max Temperature"] = city_temps
    city_input_df["Humidity"] = city_humidities
    city_input_df["Cloud Cover"] = city_clouds
    city_input_df["Wind Speed"] = city_winds
    city_input_df["Date"] = city_dates
            
    return city_input_df

In [9]:
# See function above for how I pulled data from OpenWeather API
# I just dumped to a csv after retrieving.  This is now commented out.
# I want consistent data while working with matplotlib
#my_city_weather_df = get_weather_data(my_city_df)
#my_city_weather_snapshot_csv = os.path.join("Resources", "City_Weather_Data_Snapshot.csv")
#my_city_weather_df.to_csv(my_city_weather_snapshot_csv, index=False)

# Just read in weather data previously exported
my_city_weather_snapshot_csv = os.path.join("Resources", "City_Weather_Data_Snapshot.csv")
my_city_weather_df = pd.read_csv(my_city_weather_snapshot_csv)
my_city_weather_df = my_city_weather_df.drop(columns=["URL"])
my_city_weather_df.head()

,City,Country Code,Latitude,Longitude,Name,Max Temperature,Humidity,Cloud Cover,Wind Speed,Date
0,bismil,TR,38.220664,40.589828,Turkey,84.20,35.0,0.0,1.12,1.561352e+09
1,kurtalan,TR,37.829107,41.667277,Turkey,77.00,47.0,0.0,3.36,1.561352e+09
2,sinop,TR,42.671415,35.104039,Turkey,72.00,68.0,0.0,3.36,1.561352e+09
3,akdagmadeni,TR,39.236351,36.320491,Turkey,67.51,75.0,9.0,4.72,1.561352e+09
4,solhan,TR,39.141941,41.239665,Turkey,66.20,55.0,0.0,2.80,1.561352e+09


### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

#### Latitude vs. Humidity Plot

#### Latitude vs. Cloudiness Plot

#### Latitude vs. Wind Speed Plot